In [1]:
pip install pyautogui

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install playsound


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python mediapipe numpy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pygame


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pydub


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install sounddevice numpy


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import sounddevice as sd
import time

def play_click():
    fs = 44100        # Sampling rate
    duration = 0.05   # 50ms click
    t = np.linspace(0, duration, int(fs*duration), False)
    tone = 0.5 * np.sin(2 * np.pi * 1000 * t)  # 1000Hz sine wave
    sd.play(tone, fs)
    sd.wait()  # wait until sound finishes

# Play 3 clicks in a row
for _ in range(3):
    play_click()
    time.sleep(0.1)  # short pause between clicks


In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time
import numpy as np
import sounddevice as sd
import math
import threading

# ---------------- Mediapipe Hand Detection ----------------
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# ---------------- Keyboard Layout ----------------
keys = [["Q","W","E","R","T","Y","U","I","O","P"],
        ["A","S","D","F","G","H","J","K","L"],
        ["SHIFT","Z","X","C","V","B","N","M","<","ENTER"],
        ["SPACE"]]

# ---------------- Colors & Size ----------------
key_color = (255, 0, 255)        # normal keys
key_pressed_color = (0, 255, 0)  # pressed keys
key_width, key_height = 80, 80   # key size

# ---------------- Key Tracking ----------------
last_pressed = None
debounce_time = 0.25
last_time = 0
shift_active = False

# ---------------- Programmatic click sound ----------------
def play_click():
    def beep():
        fs = 44100
        duration = 0.05
        t = np.linspace(0, duration, int(fs*duration), False)
        tone = 0.5 * np.sin(2 * np.pi * 1000 * t)
        sd.play(tone, fs)
        sd.wait()
    threading.Thread(target=beep, daemon=True).start()

# ---------------- Functions ----------------
def draw_keyboard(img):
    key_positions = {}
    x_start, y_start = 50, 100
    for row_idx, row in enumerate(keys):
        x = x_start
        for key in row:
            w = key_width
            if key == "SPACE":
                w = 6 * key_width
            elif key in ["SHIFT", "ENTER", "<"]:
                w = 2 * key_width
            y = y_start + row_idx * (key_height + 15)
            # Use normal rectangle (no borderRadius)
            cv2.rectangle(img, (x, y), (x + w, y + key_height), key_color, -1)
            cv2.putText(img, key, (x + 15, y + 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
            key_positions[key] = (x, y, w, key_height)
            x += w + 15
    return key_positions

def highlight_key(img, key_pos):
    x, y, w, h = key_pos
    overlay = img.copy()
    alpha = 0.5
    cv2.rectangle(overlay, (x, y), (x+w, y+h), key_pressed_color, -1)
    return cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)

def distance(p1, p2):
    return math.hypot(p1[0]-p2[0], p1[1]-p2[1])

# ---------------- Webcam Loop ----------------
cap = cv2.VideoCapture(0)
typed_text = ""  # optional: to display typed text on screen

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    image = cv2.flip(image, 1)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_image)
    key_positions = draw_keyboard(image)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Index fingertip (8) and thumb tip (4)
            ix, iy = int(hand_landmarks.landmark[8].x * image.shape[1]), int(hand_landmarks.landmark[8].y * image.shape[0])
            tx, ty = int(hand_landmarks.landmark[4].x * image.shape[1]), int(hand_landmarks.landmark[4].y * image.shape[0])

            cv2.circle(image, (ix, iy), 10, (0,255,0), -1)
            cv2.circle(image, (tx, ty), 10, (0,0,255), -1)
            cv2.line(image, (ix, iy), (tx, ty), (255,0,0), 2)

            # Detect click: index + thumb pinch
            if distance((ix, iy), (tx, ty)) < 40:
                for key, pos in key_positions.items():
                    kx, ky, kw, kh = pos
                    if kx < ix < kx+kw and ky < iy < ky+kh:
                        image = highlight_key(image, pos)
                        cv2.putText(image, key, (kx + 15, ky + 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

                        current_time = time.time()
                        if last_pressed != key or (current_time - last_time) > debounce_time:
                            last_time = current_time
                            last_pressed = key

                            if key == "SHIFT":
                                shift_active = not shift_active
                            elif key == "<":
                                pyautogui.press("backspace")
                            elif key == "ENTER":
                                pyautogui.press("enter")
                            elif key == "SPACE":
                                pyautogui.press("space")
                            else:
                                if shift_active:
                                    pyautogui.typewrite(key.upper())
                                    shift_active = False
                                else:
                                    pyautogui.typewrite(key.lower())

                            play_click()
                            print(f"Clicked: {key}")

    # Optional: display typed text
    cv2.rectangle(image, (50, 30), (700, 80), (50,50,50), -1)
    cv2.putText(image, typed_text, (60, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    cv2.imshow("Virtual Keyboard", image)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1756902589.798507 25603309 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1756902589.823063 25603618 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756902589.838448 25603618 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-09-03 17:59:52.079 Python[21151:25603309] +[IMKClient subclass]: chose IMKClient_Legacy
2025-09-03 17:59:52.079 Python[21151:25603309] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
/Users/asthaparoha/Library/Python/3.9/lib/python/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
 

Clicked: B
Clicked: J
Clicked: J
Clicked: F
||PaMacCore (AUHAL)|| Error on line 2796: err='-50', msg=Unknown Error
Clicked: J
||PaMacCore (AUHAL)|| Error on line 2796: err='-50', msg=Unknown Error
Clicked: J
||PaMacCore (AUHAL)|| Error on line 2796: err='-50', msg=Unknown Error
Clicked: K
Clicked: K
Clicked: K


KeyboardInterrupt: 

: 